# Zadanie domowe - łączenie danych!

Wykonujesz zlecenia dla firmy, która chce żebyśmy połączyli dane z trzech różnych źródeł. 

1. Pierwsze źródło danych to dane o klientach. Na szczęście zespół, który zajmuje się klientami, wie co robi i wszystkie dane dostępne są w bazie danych (`zad_domowe__clients.db` w tabeli `clients`).
2. Drugie źródło danych to dane o produktach. W tym przypadku dane dostępne są w pliku CSV (`zad_domowe__products.csv`).
3. Ostatnie źródło danych to dane o zamówieniach / fakturach. Tutaj niestety mamy problem. Okazuje się, że klient wszystkie faktury ma w postaci zrzutów z ekranu (pliki `zad_domowe__invoice_1.png`, `zad_domowe__invoice_2.png` etc.). 

Twoim zadaniem jest połączenie tych danych w jeden plik CSV, który będzie zawierał informacje o kliencie, produkcie i fakturze. Przygotuj plik CSV, który zawiera tylko niezbędne kolumny, nie ma żadnych kolumn powtórzonych. 

Jak rezultat wyciągania danych z faktur chcemy mieć w każdym wierszu informacje o zakupionym produkcie, ilości, kliencie, cenie jednostkowej.

Początek kodu znajdziesz w tym notebooku.

In [ ]:
# RESTARTUJ KERNEL PO URUCHOMIENIU
!pip install --quiet instructor pydantic openai

In [2]:
from pathlib import Path
import pandas as pd
import sqlite3
import base64
from getpass import getpass
from datetime import date
import instructor
from pydantic import BaseModel
from openai import OpenAI

In [3]:
# będziemy potrzebować klucza API do OpenAI żeby odczytać dane z faktur
openai_key = getpass("OpenAI key: ")

OpenAI key:  ········


## Czytamy dane klientów

In [5]:
# TUTAJ WPISZ SWÓJ KOD
#clients_df = 
with sqlite3.connect('zad_domowe__clients.db') as conn :
    clients_df = pd.read_sql('SELECT * FROM clients', conn, index_col='id')

clients_df

,name,email,phone
id,,,
1,Carla Ramsey,frypeter@example.net,(508)998-5166x12326
2,Howard Mcgee,jacobpatterson@example.org,001-669-353-6118x978
3,Eric Moore,lejacob@example.org,923-681-4646
4,Laura Beard,westdominique@example.org,(291)558-0966
5,Joshua Santiago,sheilamedina@example.com,+1-929-417-0571x74051
...,...,...,...
96,Heather Ortega,xmartin@example.org,+1-355-627-6736x91833
97,Olivia Barnes,dpacheco@example.net,9317151015
98,Tara Cabrera,geraldjames@example.net,+1-984-996-5056


## Czytamy dane produktów

In [6]:
# TUTAJ WPISZ SWÓJ KOD
products_df = pd.read_csv('zad_domowe__products.csv', sep=';', index_col='id')

products_df

,name,price
id,,
1,telewizor,85
2,laptop,47
3,smartfon,60
4,tablet,40
5,słuchawki,40
6,kamera,92
7,router,48
8,myszka,30
9,klawiatura,65


## Czytamy dane faktur

In [7]:
DATA_PATH = Path('.')

In [8]:
# na szczęście mamy już strukturę którą chcemy pobrać z faktur
class InvoiceInfoItem(BaseModel):
    description: str
    product_id: int
    quantity: int
    price: float


class InvoiceInfo(BaseModel):
    company_name: str
    customer_id: int
    customer_name: str
    invoice_number: int
    date: date
    # mamy tutaj do czynienia z listą elementów, więc danymi zagnieżdżonymi!
    items: list[InvoiceInfoItem]


instructor_openai_client = instructor.from_openai(OpenAI(api_key=openai_key))


In [9]:
#for image_path in DATA_PATH.glob("*.png"):
 #   print(f"Processing {image_path}")
orders = []
invoices = []
for image_path in sorted(DATA_PATH.glob("zad_domowe__invoice_*.png")):
    print(f"Processing {image_path}")
    with open(image_path, "rb") as file:
        image_data = base64.b64encode(file.read()).decode('utf-8')
    encoded_invoice = f"data:image/png;base64,{image_data}"


    invoice_info = instructor_openai_client.chat.completions.create(
        model="gpt-4o",
        response_model=InvoiceInfo,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Pobierz szczegóły faktury",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": encoded_invoice,
                            "detail": "high"
                        },
                    },
                ],
            },
        ],
    )
#      invoice_data = invoice_info.model_dump()

    orders.append(invoice_info.model_dump())
    #invoice_data = invoice_info.model_dump()
#orders
 #   with open(PROCESSED_DATA_PATH / f"{image_path.stem}.json", "w") as f:
  #      f.write(gas_bill.model_dump_json())

Processing zad_domowe__invoice_1.png
Processing zad_domowe__invoice_10.png
Processing zad_domowe__invoice_2.png
Processing zad_domowe__invoice_3.png
Processing zad_domowe__invoice_4.png
Processing zad_domowe__invoice_5.png
Processing zad_domowe__invoice_6.png
Processing zad_domowe__invoice_7.png
Processing zad_domowe__invoice_8.png
Processing zad_domowe__invoice_9.png


In [20]:
flat_orders = [
    {
        "company_name": order["company_name"],
        "customer_id": order["customer_id"],
        "customer_name": order["customer_name"],
        "invoice_number": order["invoice_number"],
        "date": order["date"],
        **item  # rozpakowanie słownika z item
    }
    for order in orders
    for item in order["items"]
]

In [22]:
orders_df = pd.DataFrame(flat_orders)
orders_df

,company_name,customer_id,customer_name,invoice_number,date,description,product_id,quantity,price
0,Firma XYZ,3,Eric Moore,2,2024-05-25,router,7,3,48.0
1,Firma XYZ,3,Eric Moore,2,2024-05-25,narty,17,5,55.0
2,Firma XYZ,92,Valerie Stevens,11,2024-05-02,słuchawki,5,1,40.0
3,Firma XYZ,60,Troy Alexander,3,2024-05-19,ochraniacze,16,4,46.0
4,Firma XYZ,78,William Velazquez,4,2024-03-26,rolki,14,3,11.0
5,Firma XYZ,33,Steven Jackson,5,2024-07-23,pendrive,10,4,47.0
6,Firma XYZ,33,Steven Jackson,5,2024-07-23,laptop,2,2,47.0
7,Firma XYZ,8,Ronald Callahan,6,2024-03-14,słuchawki,5,4,40.0
8,Firma XYZ,8,Ronald Callahan,6,2024-03-14,ochraniacze,16,2,46.0
9,Firma XYZ,4,Laura Beard,7,2024-06-24,telewizor,1,2,85.0


## Łączenie danych

In [27]:
# --- Połączenie z klientami (LEFT JOIN po ID klienta)
merged_df = orders_df.merge(
    clients_df,
    how="left",
    left_on="customer_id",
    right_on="id",
    suffixes=("", "_client")
)

# --- Połączenie z produktami (LEFT JOIN po ID produktu)
merged_df = merged_df.merge(
    products_df,
    how="left",
    left_on="product_id",
    right_on="id",
    suffixes=("", "_product")
)

# --- Wybór tylko potrzebnych kolumn (bez duplikatów)
company_df = merged_df[[
    "company_name",
    "invoice_number",
    "date",
    "customer_id",
    "customer_name",
    "email",
    "phone",
    "description",       # z orders
    "product_id",
    "quantity",
    "price"
]]

In [28]:
# --- Zapis do CSV
company_df.to_csv("zadanie_domowe_modul5_wynik_PK.csv",  sep=';', index=False)

In [29]:
company_df

,company_name,invoice_number,date,customer_id,customer_name,email,phone,description,product_id,quantity,price
0,Firma XYZ,2,2024-05-25,3,Eric Moore,lejacob@example.org,923-681-4646,router,7,3,48.0
1,Firma XYZ,2,2024-05-25,3,Eric Moore,lejacob@example.org,923-681-4646,narty,17,5,55.0
2,Firma XYZ,11,2024-05-02,92,Valerie Stevens,xlowe@example.com,757-943-1579x98346,słuchawki,5,1,40.0
3,Firma XYZ,3,2024-05-19,60,Troy Alexander,nturner@example.net,+1-212-349-3992x430,ochraniacze,16,4,46.0
4,Firma XYZ,4,2024-03-26,78,William Velazquez,lisa25@example.net,(765)602-1572x949,rolki,14,3,11.0
5,Firma XYZ,5,2024-07-23,33,Steven Jackson,jeremy07@example.com,481.258.2434,pendrive,10,4,47.0
6,Firma XYZ,5,2024-07-23,33,Steven Jackson,jeremy07@example.com,481.258.2434,laptop,2,2,47.0
7,Firma XYZ,6,2024-03-14,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,słuchawki,5,4,40.0
8,Firma XYZ,6,2024-03-14,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,ochraniacze,16,2,46.0
9,Firma XYZ,7,2024-06-24,4,Laura Beard,westdominique@example.org,(291)558-0966,telewizor,1,2,85.0
